In [41]:
import json
import numpy 
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import pyarrow
import sys
from tqdm import tqdm

from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine
from os import path
from typing import List,Dict, Tuple
from collections import defaultdict
pd.set_option("display.max_columns", None)

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../scripts")))

import create_queries
import prepare_data

from bespoke.inventory.analysis.shared import download_util, inventory_types
from bespoke.inventory.analysis import active_inventory_util as util
from bespoke.inventory.analysis import inventory_valuations_util as valuations_util

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from underwriting import data_quality_checks

In [3]:
TRANSFER_PACKAGES_START_DATE = '2020-01-01'
SALES_TRANSACTIONS_START_DATE = '2020-01-01'
CURRENT_MONTH = '2022-01'

In [10]:
company_identifier_list = ['RA','HPCC','SFVPC','EMA','EMF','EMM','EMT','GRG','ST','EL']

In [8]:
def fetch_data(company_identifier,transfer_packages_start_date,sales_transactrions_start_date,current_month):
    df_in,df_sales_deduped,bad_dl,unknown_transfer,rwp,miss_incoming,miss_receipts = data_quality_checks.run(company_identifier,transfer_packages_start_date,sales_transactrions_start_date)    
    return df_in,df_sales_deduped
    

In [11]:
###test out
incoming_transfer_data,sales_transaction_deduped = fetch_data(['EL'],TRANSFER_PACKAGES_START_DATE,SALES_TRANSACTIONS_START_DATE,CURRENT_MONTH)


Verifying download summaries for license C9-0000370-LIC...
Earliest download summary: 2020-01-01
Latest download summary: 2022-02-15

[SUCCESS] All download summaries look good!
[SUCCESS] No unknown transfer packages!
97.0% of incoming transfer packages have receiver wholesale price
# transactions missing incoming transfer package: 22 (0.11358356136093757%)
# transactions total: 19369
# receipts with mismatching transactions: 0 (0.0%)
# receipts missing transactions: 0 (0.0%)
# receipts total: 7569


In [30]:
sales_transaction_deduped['tx_unit_of_measure'].unique()

array(['Each', 'Grams'], dtype=object)

In [15]:
sales_transaction_deduped.head()

,license_number,receipt_number,rt_type,sales_customer_type,sales_datetime,sales_month,total_packages,rt_total_price,tx_type,tx_package_id,tx_package_label,tx_product_name,tx_product_category_name,tx_unit_of_measure,tx_quantity_sold,tx_total_price
0,C9-0000370-LIC,0165026072,active,Consumer,2022-02-14 20:52:42+00:00,2022-02,2,60.0,active,21023428,1A406030002B8E1000006961,FPB150-B8e,Flower (packaged eighth - each),Each,1.0,30.0
1,C9-0000370-LIC,0165026072,active,Consumer,2022-02-14 20:52:42+00:00,2022-02,2,60.0,active,21023431,1A406030002B8E1000006963,GMO59-B8e,Flower (packaged eighth - each),Each,1.0,30.0
2,C9-0000370-LIC,0165026071,active,Consumer,2022-02-14 19:47:48+00:00,2022-02,4,205.0,active,20358475,1A4060300022B79000204403,"Buddies 10:10 THC/CBD Capsule 50pc , Hashtag D...",Capsule (weight - each),Each,1.0,60.0
3,C9-0000370-LIC,0165026071,active,Consumer,2022-02-14 19:47:48+00:00,2022-02,4,205.0,active,18181096,1A40603000092E5000286522,Almora Farm: 1.2g Jar: Sugar: Sour Berry [H],Other Concentrate (weight - each),Each,2.0,70.0
4,C9-0000370-LIC,0165026071,active,Consumer,2022-02-14 19:47:48+00:00,2022-02,4,205.0,active,21841382,1A406030000A429000324130,STIIIZY - Blue Dream Curated Live Resin Sauce ...,Extract (weight - each),Each,1.0,40.0


In [42]:
sales_transaction_deduped['sales_customer_type'].unique()

array(['Consumer', 'Patient'], dtype=object)

In [29]:
sales_transaction_deduped[sales_transaction_deduped['tx_unit_of_measure'] == 'Grams']['tx_product_category_name'].unique()

array(['Flower'], dtype=object)

In [31]:
sales_transaction_deduped[sales_transaction_deduped['tx_unit_of_measure'] == 'Each']['tx_product_category_name'].unique()

array(['Flower (packaged eighth - each)', 'Capsule (weight - each)',
       'Other Concentrate (weight - each)', 'Extract (weight - each)',
       'Flower (packaged ounce - each)', 'Pre-Roll Flower',
       'Edible (weight - each)', 'Topical (volume - each)',
       'Tincture (volume - each)', 'Vape Cartridge (weight - each)',
       'Flower (packaged quarter - each)',
       'Flower (packaged gram - each)',
       'Flower (packaged half ounce - each)', 'Pre-Roll Infused',
       'Flower (packaged - each)', 'Pre-Roll Leaf',
       'Vape Cartridge (volume - each)', 'Extract (volume - each)',
       'Tincture (weight - each)', 'Shake (Packaged Half Ounce - each)',
       'Topical (weight - each)', 'Edible (volume - each)'], dtype=object)

In [38]:
incoming_transfer_data.loc[3]['product_name']

'Highrize - Pineapple Cheese Cake Premium Flower 12pk Tins'

In [39]:
incoming_transfer_data.loc[2]['product_name']

'Highrize - Octane Kush Premium Flower 12pk Tins'

In [40]:
incoming_transfer_data

,delivery_type,license_number,manifest_number,created_date,received_datetime,shipment_transaction_type,shipper_facility_license_number,shipper_facility_name,recipient_facility_license_number,recipient_facility_name,shipment_type_name,shipment_transaction_type_1,package_id,package_label,type,source_package_labels,source_harvest_names,shipment_package_state,is_testing_sample,is_trade_sample,product_category_name,product_name,package_lab_results_status,shipper_wholesale_price,shipped_quantity,shipped_unit_of_measure,receiver_wholesale_price,received_quantity,received_unit_of_measure,receiver_wholesale_price_1,item_unit_weight,item_unit_weight_unit_of_measure_name
0,INCOMING_FROM_VENDOR,C9-0000370-LIC,0003076021,2022-02-11,NaT,Wholesale,C11-0000962-LIC,"965 SOLUTIONS, LLC",C9-0000370-LIC,"ELITE HERBS, INC",Wholesale Manifest,Wholesale,22209162,1A406030000902D000012992,transfer,1A406030000902D000009367,"2019-08-02_Royal Ringo_GH3 BED B, 2019-08-15_R...",Shipped,False,False,Pre-Roll Flower,Highrize - Royal Ringo Premium Flower 12 pack ...,passed,180.00,20.0,Each,NaN,NaN,None,NaN,4.2,Grams
1,INCOMING_FROM_VENDOR,C9-0000370-LIC,0003076021,2022-02-11,NaT,Wholesale,C11-0000962-LIC,"965 SOLUTIONS, LLC",C9-0000370-LIC,"ELITE HERBS, INC",Wholesale Manifest,Wholesale,22208898,1A406030000902D000012984,transfer,1A406030000902D000010362,"20HGF(ranch)rm9bed11GMO, GF100120, GF101220, G...",Shipped,False,False,Pre-Roll Flower,Highrize - Dr. Funkentine Premium Flower Singl...,passed,450.00,150.0,Each,NaN,NaN,None,NaN,1.0,Grams
2,INCOMING_FROM_VENDOR,C9-0000370-LIC,0003076021,2022-02-11,NaT,Wholesale,C11-0000962-LIC,"965 SOLUTIONS, LLC",C9-0000370-LIC,"ELITE HERBS, INC",Wholesale Manifest,Wholesale,22209139,1A406030000902D000012988,transfer,1A406030000902D000011367,20HGF(ranch)rm9bed11GMO,Shipped,False,False,Pre-Roll Flower,Highrize - Octane Kush Premium Flower 12pk Tins,passed,360.00,20.0,Each,NaN,NaN,None,NaN,4.0,Grams
3,INCOMING_FROM_VENDOR,C9-0000370-LIC,0003076021,2022-02-11,NaT,Wholesale,C11-0000962-LIC,"965 SOLUTIONS, LLC",C9-0000370-LIC,"ELITE HERBS, INC",Wholesale Manifest,Wholesale,22209143,1A406030000902D000012989,transfer,1A4060300002F45000002153,"PC harvest 2020, POG HVST 2020, SC Box 3 LD ha...",Shipped,False,False,Pre-Roll Flower,Highrize - Pineapple Cheese Cake Premium Flowe...,passed,180.00,20.0,Each,NaN,NaN,None,NaN,4.0,Grams
4,INCOMING_FROM_VENDOR,C9-0000370-LIC,0003076021,2022-02-11,NaT,Wholesale,C11-0000962-LIC,"965 SOLUTIONS, LLC",C9-0000370-LIC,"ELITE HERBS, INC",Wholesale Manifest,Wholesale,22209164,1A406030000902D000012993,transfer,1A406030000902D000009367,"2019-08-02_Royal Ringo_GH3 BED B, 2019-08-15_R...",Shipped,False,False,Pre-Roll Flower,Highrize - Royal Ringo Premium Flower 12 pack ...,passed,0.20,20.0,Each,NaN,NaN,None,NaN,4.2,Grams
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1948,INCOMING_FROM_VENDOR,C9-0000370-LIC,0000966581,2020-09-03,2020-09-09 00:23:45+00:00,Wholesale,C12-0000057-LIC,SUNDERSTORM BAY LLC,C9-0000370-LIC,"ELITE HERBS, INC",Wholesale Manifest,Wholesale,7628369,1A4060300006E2E000042538,transfer,1A406030000296B000059853,"1st, 2019-10-18-2-H, 2019-10-18-3-H, 2019-10-1...",Accepted,False,False,Edible (weight - each),KE - THC Indica Strawberry 100mg - 43g,passed,0.01,1.0,Each,0.01,1.0,Each,0.01,43.0,Grams
1949,INCOMING_FROM_VENDOR,C9-0000370-LIC,0000966581,2020-09-03,2020-09-09 00:23:45+00:00,Wholesale,C12-0000057-LIC,SUNDERSTORM BAY LLC,C9-0000370-LIC,"ELITE HERBS, INC",Wholesale Manifest,Wholesale,7628137,1A4060300006E2E000042539,transfer,1A406030000296B000059349,"1st, 2019-10-18-2-H, 2019-10-18-3-H, 2019-10-1...",Accepted,False,False,Edible (weight - each),KE - THC Hybrid Watermelon 100mg - 43g,passed,0.01,1.0,Each,0.01,1.0,Each,0.01,43.0,Grams
1950,INCOMING_FROM_VENDOR,C9-0000370-LIC,0000966581,2020-09-03,2020-09-09 00:23:45+00:00,Wholesale,C12-0000057-LIC,SUNDERSTORM BAY LLC,C9-0000370-LIC,"ELITE HERBS, INC",Whol